# Kurze Einführung in Tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

In [ ]:
# Initialisieren mit Einern
x = tf.ones(shape=(2,1))

In [ ]:
x

In [ ]:
# Initialisieren mit Nullen
x = tf.zeros(shape=(2,1))

In [ ]:
print(x)

In [ ]:
# Initialisieren mit gaußverteilten Zufallszahlen
x = tf.random.normal(shape=(3,1),mean=0.,stddev=1.0)

In [ ]:
tf.print(x)

In [ ]:
x?

In [ ]:
# Grundrechenarten
a = tf.ones((2, 2)) 
b = tf.square(a)
c = tf.sqrt(a) 
d = b + c
#e = tf.matmul(a, b) # normale matrix multiplication
e = a@b # fancy mat mul
e *= d # elementwise

In [ ]:
# Assignment geht in Numpy
import numpy as np
x = np.ones(shape=(2,2))
x[0,0] = 0.

In [ ]:
# Aber nicht in Tensorflow - diese Tensoren sind Konstanten
# Wie in Mathe - wenn x = 1 dann kann nicht x = 3 sein. Neu definieren geht (wie oben), aber kein Assignment
x = tf.ones(shape=(2,2))
x[0,0] = 0.

In [ ]:
# Um Tensoren ändern zu können, müssen wir Variablen definieren
v = tf.Variable(initial_value=tf.random.normal(shape=(3,1)))
print(v)

In [ ]:
vv = v+np.ones(shape=(3,1))
vv

In [ ]:
# Jetzt kann man assignen, auch über .assign:
v.assign(tf.ones((3, 1)))

In [ ]:
# Etwas dazu addieren ...
v.assign_add(tf.ones((3, 1)))

In [ ]:
v = tf.zeros((3,1))  # Vorsicht: Jetzt änderst Du v in eine Konstante!

In [ ]:
v.assign_add(tf.ones((3, 1)))

In [ ]:
# Gradienten berechnen: gradients = tape.gradient(loss, weights)
# Innerhalb des "Gradient Tape"-Scope werden Operationen aufgezeichnet, falls ein Input "beobachtet" wird
# tf.Variable werden immer beobachtet, weil der Sinn und Zweck von tf ja diese Gradientenberechnung ist.
# Die Gradienten der "aufgezeichneten" Operationen können automatisch berechnet werden.

input_var = tf.Variable(initial_value=3.)


with tf.GradientTape() as tape:     # Der Gradient Tape wird gestartet
    result = tf.square(input_var)   # Result wird aufgezeichnet
    
gradient = tape.gradient(result, input_var) # Wir können den Gradienten von result bezüglich input_var berechnen


In [ ]:
gradient  # x^2 abgeleitet ist 2x an der Stelle 3 = 6

# Übung

* Beschleunigung berechnen für etwas, was sich mit 4.9 * time^2 vorwärts bewegt.
* Beschleunigung ist die Ableitung der Geschwindigkeit 
* Geschwindigkeit ist die Ableitung der Vorwärtsbewegung


# Übung2
Wenn die Verzweiflung groß ist, dann macht erstmal diese einfacheren Sachen:
* Ableitung von der Wurzelfunktion berechnen an den Punkten 1, 0, -1
* Wo geht das schief? Geht das schief?
* Gradienten berechnen einer Funktion mit Tensorwertigem Input z.B. shape=(2,2)
* Vielleicht ein Dotprodukt mit sich selbst.

In [ ]:
wurzel_var = tf.Variable(initial_value=0.)
with tf.GradientTape() as tape: 
    result = tf.sqrt(wurzel_var)
gradient = tape.gradient(result, wurzel_var)

In [ ]:
gradient

In [ ]:
matrix = tf.Variable(shape=(2,2), initial_value = tf.random.normal(shape=(2,2)))
with tf.GradientTape() as tape: 
    result = matrix@matrix
gradient = tape.gradient(result, matrix)

In [ ]:
gradient

In [ ]:
# Übung: Acceleration berechnen mit Gradient Tape:

time = tf.Variable(0.)
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape: 
        position = 4.9 * time ** 2
    speed = inner_tape.gradient(position, time) 
acceleration = outer_tape.gradient(speed, time)

In [ ]:
tf.print(acceleration)

# Einen linearen Classifier mit Tensorflow bauen

In [ ]:
# Daten generieren:

num_samples_per_class = 1000

negative_samples = np.random.multivariate_normal(
    mean=[0, 3], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class) 

positive_samples = np.random.multivariate_normal(
    mean=[3, 0], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class)

In [ ]:
# Trainingsdaten: (vstack ist vertical stack, also vertikal aufeinanderstapeln --> entlang der Reihe)

inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype='float32'), np.ones((num_samples_per_class, 1), dtype='float32')))

In [ ]:
inputs2 = np.concatenate([negative_samples, positive_samples],axis=0).astype(np.float32)

In [ ]:
(inputs == inputs2).all().all()

In [ ]:
# Visualisieren:
%matplotlib inline
import matplotlib.pyplot as plt
plt.scatter(inputs[:, 0], inputs[:, 1], c=targets[:, 0]) 
plt.figure(figsize=(10,10))
plt.show()

In [ ]:
# Die Variablen

input_dim = 2
output_dim = 1

W = tf.Variable(initial_value=tf.random.uniform(shape=(input_dim, output_dim))) 
b = tf.Variable(initial_value=tf.zeros(shape=(output_dim,)))

In [ ]:
# Forward Pass Funktion
def model(inputs):
    return tf.matmul(inputs, W) + b

In [ ]:
# Loss Funktion
def square_loss(targets, predictions): 
    per_sample_losses = tf.square(targets - predictions) 
    return tf.reduce_mean(per_sample_losses)

In [ ]:
# Trainingstep
learning_rate = 0.1

def training_step(inputs, targets): 
    
    # Der Gradient
    with tf.GradientTape() as tape:     
        predictions = model(inputs) # Forward Pass
        loss = square_loss(predictions, targets)  # Loss dieser Vorhersagen
    grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W, b]) 
        
    # Der Update-Schritt - Gradient wird subtrahiert von den Weights
    W.assign_sub(grad_loss_wrt_W * learning_rate)
    b.assign_sub(grad_loss_wrt_b * learning_rate)
    
    return loss

In [ ]:
# Das Training
for step in range(20):
    loss = training_step(inputs, targets) 
    print('Loss at step %d: %.4f' % (step, loss))

In [ ]:
# Das Ergebnis
predictions = model(inputs)
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5) 
plt.show()

In [ ]:
x = np.linspace(-1, 4, 100)  # gleichmäßig gesampelte x-Werte 
y = - W[0] / W[1] * x + (0.5 - b) / W[1]  # y-Werte 
plt.plot(x, y, '-r')
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5)
plt.show()

In [ ]:
W

# Übung:
* Diese Klassifizierung ist ja nicht ideal
* Versucht ein neues Modell zu trainieren, dass ein Testset besser aufteilt
* Niedrigster Loss auf einem Testset gewinnt
